In [5]:
import torch
import time
import argparse
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import model

from custom_data_loader import *
from main import *

usage: ipykernel_launcher.py [-h] [--output_dir OUTPUT_DIR]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9018 --control=9016 --hb=9015 --Session.signature_scheme="hmac-sha256" --Session.key=b"b446a990-2002-428d-a7a6-ff8b7dad97c4" --shell=9017 --transport="tcp" --iopub=9019 --f=/uufs/chpc.utah.edu/common/home/u1471428/.local/share/jupyter/runtime/kernel-v2-2327565UqsIJXW28Aip.json


SystemExit: 2

In [6]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Pytorch version: {torch.__version__}")
    print(f"Device name: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("No GPU available.")

Pytorch version: 2.0.1+cu117
Device name: NVIDIA GeForce GT 1030


In [8]:
best_perf_dict = {
    "metric": -1,
    "model_param": None,
    "optim_param": None,
    "epoch": 0,
    "learning_rate": 0,
}

In [7]:
torch.manual_seed(42)
learning_rates = [0.01]  # , 0.001, 0.0001]

seq_len = 500

train_folder = "./mp3_release/data/train"
dev_folder = "./mp3_zrelease/data/dev"
test_folder = "./mp3_release/data/test"



In [72]:
vocab, rev_vocab = getVocab("./mp3_release/data/vocab.pkl")
ignore_index = vocab["[PAD]"]

In [7]:
train_char_data = getCharData(train_folder, vocab)
# dev_char_data = getCharData(dev_folder, vocab)
test_char_data = getCharData(test_folder, vocab)

In [8]:
weights = getWeights(vocab, train_char_data)

In [ ]:
train_dataset = getData(vocab, train_char_data, 500)
dev_dataset = getData(vocab, dev_char_data, 500)

879963


: 

In [ ]:
test_dataset = getData(vocab, test_char_data,500)

154145
torch.Size([167803, 500])
torch.Size([167803, 500])


In [9]:
test_model = model.LMModel(len(vocab), num_layers=2)

In [9]:
def loadModel(model_instance, model_path):
    checkpoint = torch.load(model_path)
    # print(checkpoint)
    model_instance.load_state_dict(checkpoint["model_param"])
    print(
        f"""Dev_LAS of loaded model: {checkpoint["dev_metric"]} at epoch {checkpoint["epoch"]} with learning rate {checkpoint["learning_rate"]}"""
    )
    return model_instance

In [9]:
best_model_path = "/scratch/general/vast/u1471428/cs6957/assignment3/models/layer_2/0.01_0"
best_model = loadModel(test_model, best_model_path)

Dev_LAS of loaded model: 1.3527647256851196 at epoch 0 with learning rate 0.01


In [10]:
num_param = sum( p . numel () for p in test_model.parameters () )

print(num_param)

718986


In [20]:
def evaluateModel(model, dataset):
    model = model.to(device)

    dataLoader = torch.utils.data.DataLoader(
            dataset, 128
        )
    with torch.no_grad():
        loss_function = nn.CrossEntropyLoss(
            weight=weights, ignore_index=ignore_index, reduce=False
        )

        loss_function = loss_function.to(device)
        model.eval()
        perplexity = 0

        total_len = 0
        for input, target in dataLoader:
            batch_size = input.shape[0]
            total_len += batch_size
            seq_len = input.shape[1]

            output = model(input.to(device))
            # sen_len * batch_size * vocab_size
            # print("output ", output.shape)
            # target = target.permute(1, 0)
            # print("target", target.shape)

            output = output.view(-1, output.shape[-1])
            target = target.reshape(-1)

            char_level_loss = loss_function(
                output, target.to(device)
            )  # seq_len x batch_size

            # print(char_level_loss.shape)

            for i in range(batch_size):
                seq_loss = 0
                for j in range(i * seq_len, (i + 1) * seq_len):
                    seq_loss += char_level_loss[j]

                seq_loss = seq_loss / seq_len
                perplexity += 2**seq_loss
            del char_level_loss

        perplexity = perplexity / total_len

        print(f"Perplexity on test dataset: {perplexity}")


In [21]:
evaluateModel(test_model, test_dataset)

/uufs/chpc.utah.edu/common/home/u1471428/miniconda3/envs/asg/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


Perplexity on test dataset: 1.3065770864486694


: 

In [ ]:
start_time = time.time()

for lr in learning_rates:
    print("*****************")
    print("learning_rate", lr)
    trainer = Trainer(
        model1,
        train_dataset,
        dev_dataset,
        weights,
        ignore_index,
        batch_size=128,
        epochs=5,
    )
    trainer.trainModel(lr)
    print("*****************")
    print()

end_time = time.time()
print("--- %s seconds ---" % (end_time - start_time))

In [101]:
sentences = [
    "The little boy was",
    "Once upon a time in",
    "With the target in",
    "Capitals are big cities. For example,",
    "A cheap alternative to",
    "Salt Lake City in Utah is filled with Hobos",
    "Johnny Johnny yes papa"
]

In [102]:
from custom_utils import convert_sent2idx
sent_data = convert_sent2idx(sentences,vocab)

In [103]:
tensor_sent_data = []
for data in sent_data:
    tensor_data = torch.tensor(data)
    tensor_data = torch.unsqueeze(tensor_data,dim=0)
    tensor_sent_data.append(tensor_data)

print(tensor_sent_data[0].shape)
    

torch.Size([1, 18])


In [104]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Pytorch version: {torch.__version__}")
    print(f"Device name: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("No GPU available.")


class LMModel2(nn.Module):
    def __init__(self, vocab_dim, emb_dim=50, hidden_dim=200, num_layers=1):
        super(LMModel2, self).__init__()
        self.vocab_dim = vocab_dim
        self.emb_dim = emb_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.projection_layer = nn.Embedding(vocab_dim, emb_dim)

        self.lstm = nn.LSTM(
            input_size=self.emb_dim,
            hidden_size=self.hidden_dim,
            num_layers=self.num_layers,
            batch_first=True,
        )

        self.fc_layer = nn.Sequential(
            nn.Linear(hidden_dim, 300, bias=True),
            nn.ReLU(),
            nn.Linear(300, self.vocab_dim, bias=True),
        )

    def forward(self, input):
        # print(input.shape)
        self.batch_size = input.shape[0]
        for i in range(200):
            input_emb = self.projection_layer(input)

            

            h_0 = Variable(
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim).to(device)
            )
            c_0 = Variable(
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim).to(device)
            )

            lstm_output, hidden = self.lstm(input_emb, (h_0, c_0))
            # print(lstm_output.shape)

            next_char_prob = self.fc_layer(lstm_output[:,-1,:])

            # print(next_char_prob.shape)
            # print(next_char_prob)

            next_char_prob = nn.Softmax(dim=1)(next_char_prob)

            # print(next_char_prob.shape)
            # print(next_char_prob)



            sample = torch.multinomial(next_char_prob,1)

            # print(sample)
            # print(sample.shape)

            input = torch.cat((input,sample),dim=1)
            # print(input.shape)



        return input

    # def forward_step(self, )

Pytorch version: 2.0.1+cu117
Device name: NVIDIA GeForce GT 1030


In [105]:
# from model2 import LMModel2
model_2 = LMModel2(len(vocab),num_layers=2)
best_model_path = "/scratch/general/vast/u1471428/cs6957/assignment3/models/layer_2/0.01_0"
best_model = loadModel(model_2, best_model_path)
best_model = best_model.to(device)

Dev_LAS of loaded model: 1.2572855949401855 at epoch 0 with learning rate 0.01


In [106]:
with torch.no_grad():
    best_model.eval()
    for sent in tensor_sent_data: 
        extended_input = best_model(sent.to(device))
        # print(extended_input.shape)
        extended_input = extended_input.view(-1)
        # print(extended_input.shape)
        gen_seq = ""
        for ind in extended_input:
            gen_seq += rev_vocab[ind.item()]
        print(gen_seq)
        print("****************************************")


The little boy was thank or disappanching, with the Siviente Liceivage, (the ladies in the fallance.=
 philosophy. The time dominated itself on angles found. The verulous elefficer submitter bar been proper Soul to see
****************************************
Once upon a time in his cigar collection,
 aparted with girl of musima’s datchard, with the doctor limitiestate of trusts a step down the open first that was a paperd - will establish thousted, to shight or catain and v
****************************************
With the target into the outer opposite lants followed the beam.
.ory. The Englissy of Himself, N. Hussy and Athenshillonmobirt Mit gave the seats, and an I was some thousand array, of the misery—we better in Manow, me
****************************************
Capitals are big cities. For example, and there "write.
 accepting it started leave to taggest indignation; I must be dimsy is for work in the fact you are that lack is insisted by reply out of the new Highest. The down

In [74]:
print(rev_vocab[4])